In [18]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
import numpy as np

# Input data params
img_height = 28
img_width = 28
img_channels = 1
num_classes = 10

# Training params
learning_rate = 0.0001#学习率
batch_size = 100
num_epochs = 1000
# Get the data
#mnist_data = input_data.read_data_sets('data/fashion/', one_hot = True)

# Model
layer1_size = 5
layer1_stride = 1
layer1_filters = 8

layer2_size = 4
layer2_stride = 2
layer2_filters = 12

layer3_size = 4
layer3_stride = 2
layer3_filters = 24

layer4_nodes = 200

# Input layer
X = tf.placeholder(tf.float32, [None, img_height * img_width])
X_img = tf.reshape(X, [-1, img_width, img_height, img_channels])
Y_ = tf.placeholder(tf.float32, [None, num_classes])

# Layer 1
W1 = tf.Variable(tf.truncated_normal([layer1_size, layer1_size, img_channels, layer1_filters], stddev = 0.1))
b1 = tf.Variable(tf.ones([layer1_filters]) / 10)
Y1 = tf.nn.relu(tf.nn.conv2d(X_img, W1, strides = [1, layer1_stride, layer1_stride, 1], padding = 'SAME') + b1)

# Layer 2
W2 = tf.Variable(tf.truncated_normal([layer2_size, layer2_size, layer1_filters, layer2_filters], stddev = 0.1))
b2 = tf.Variable(tf.ones([layer2_filters]) / 10)
Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides = [1, layer1_stride, layer1_stride, 1], padding = 'SAME') + b2)

# Layer 3
W3 = tf.Variable(tf.truncated_normal([layer3_size, layer3_size, layer2_filters, layer3_filters], stddev = 0.1))
b3 = tf.Variable(tf.ones([layer3_filters]) / 10)
Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides = [1, layer3_stride, layer3_stride, 1], padding = 'SAME') + b3)
Y3_shape = Y3.get_shape()
# print Y3_shape
Y3_flat = tf.reshape(Y3, [-1, 14 * 14 * 24])#tf.reshape(Y3, [-1, (Y3_shape[1] / (layer1_stride * layer2_stride)) * (Y3_shape[2] / (layer1_stride * layer2_stride)) * Y3_shape[3]])

# Layer 4
W4 = tf.Variable(tf.truncated_normal([14 * 14 * 24, layer4_nodes], stddev = 0.1))#tf.Variable(tf.truncated_normal([Y3_shape[0] * Y3_shape[1] * Y3_shape[2], layer4_nodes], stddev = 0.1))
b4 = tf.Variable(tf.ones([layer4_nodes]) / 10)
Y4 = tf.nn.relu(tf.matmul(Y3_flat, W4) + b4)

# Layer 5
W5 = tf.Variable(tf.truncated_normal([layer4_nodes, num_classes], stddev = 0.1))
b5 = tf.Variable(tf.zeros([num_classes]))
Y = tf.nn.softmax(tf.matmul(Y4, W5) + b5)

# Loss function - Cross Entropy
cross_entropy = - tf.reduce_sum(Y_ * tf.log(Y))

# % of correct answers found in batch
is_correct = tf.equal(tf.argmax(Y, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(cross_entropy)

nn_txt = "./nn_txt"
if not os.path.exists(nn_txt):
    os.makedirs(nn_txt)

saver = tf.train.Saver()
# Create the session
sess = tf.Session()

# Initialize the variables
sess.run(tf.global_variables_initializer())
txt = tf.train.get_checkpoint_state(nn_txt)
if txt and txt.model_checkpoint_path:
    print(txt.model_checkpoint_path)
    saver.restore(sess, txt.model_checkpoint_path)
# Run our Model!!
# for i in range(num_epochs):
#     # Load the input data
#     batch_X, batch_Y = mnist_data.train.next_batch(batch_size)
#     train_data = {X: batch_X, Y_: batch_Y}

#     # Train
#     sess.run(train_step, feed_dict = train_data)

#     # Accuracy on training data
#     acc, loss = sess.run([accuracy, cross_entropy], feed_dict = train_data)

#     # Trying it out on test data
#     test_data = {X: mnist_data.test.images, Y_: mnist_data.test.labels}

#     # Accuracy on test data
#     acc_test, loss_test = sess.run([accuracy, cross_entropy], feed_dict = test_data)
#     saver.save(sess,nn_txt + "/model.ckpt")
#     print ("acc:%4f"%acc)
class Dataset(object):
    def __init__(self, dtype='uint8', is_row_iamge=False):
        images = np.fromfile('data/test images/images/test_image.bin', dtype=np.uint8).reshape(-1, 28, 28, 1)
        print(images.shape)
        if dtype == 'uint8':
            self.images = images
        else:
            images = images.astype(np.float32) / 255.
            self.images = images
        if is_row_iamge:
            self.images = images.reshape([-1, 784])
        self.num_of_images = 6500
        self.offset = 0
        print('共6500张图片')
        
        
    def next_batch(self, batch_size=50):
        # 返回False表示以及没有样本
        # 注意：最后一个批次可能不足batch_size 所以推荐选择6500可以整除的batch_size
        if (self.offset + batch_size) <= self.num_of_images:
            return self.images[self.offset-batch_size : self.offset]
        elif self.offset < self.num_of_images:
            self.offset += batch_size
            return self.images[self.offset : ]
        else:
            False
images = Dataset(is_row_iamge=True).images
#batch = images.next_batch()
with tf.Session() as sess:
    sess.run([tf.local_variables_initializer(), tf.global_variables_initializer()])
    res_label = sess.run(Y,feed_dict = {X:images})
    print(np.argmax(res_label,1))
    

./nn_txt/model.ckpt
INFO:tensorflow:Restoring parameters from ./nn_txt/model.ckpt


INFO:tensorflow:Restoring parameters from ./nn_txt/model.ckpt


(6500, 28, 28, 1)
共6500张图片
[4 4 4 ..., 4 4 4]


In [3]:
np.argmax(res_label,1).shape

(6500,)

In [17]:
c = 0
with open("data/Gaoyanzhao.txt","w") as f:
    for i in range(np.argmax(res_label,1).shape[0]):
        if np.argmax(res_label,1)[i] == 3:
            c += 1
        f.write(str(np.argmax(res_label,1)[i])+'\n')
c

228

In [1]:
images = Dataset(is_row_iamge=True)

NameError: name 'Dataset' is not defined

In [9]:
res =images.next_batch(50)

In [10]:
res.shape

(0, 784)